<a href="https://colab.research.google.com/github/yasohasakii/kaggle_histopathologic_cancer_detection/blob/master/cancer_detection_using_Nasnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/yasohasakii/kaggle_histopathologic_cancer_detection.git
!pip install kaggle
!cp kaggle_histopathologic_cancer_detection/kaggle.json ~/.kaggle/kaggle.json
!rm -rf kaggle_histopathologic_cancer_detection sample_data
!kaggle competitions download -c histopathologic-cancer-detection

Cloning into 'kaggle_histopathologic_cancer_detection'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.
kaggle_histopathologic_cancer_detection  sample_data


In [0]:
!unzip test.zip

In [0]:
!kaggle competitions download -c histopathologic-cancer-detection

  0% 0.00/1.33M [00:00<?, ?B/s]
100% 1.33M/1.33M [00:00<00:00, 89.3MB/s]
  0% 0.00/5.10M [00:00<?, ?B/s]
100% 5.10M/5.10M [00:00<00:00, 46.9MB/s]
 99% 1.29G/1.30G [00:09<00:00, 67.9MB/s]
100% 1.30G/1.30G [00:09<00:00, 141MB/s] 
100% 4.98G/4.98G [02:53<00:00, 24.0MB/s]
100% 4.98G/4.98G [02:53<00:00, 30.8MB/s]
